In [ ]:
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

import sys
sys.path.append('src')

from src.inference import translate_with_idioms
from src.evaluation import evaluate_model, print_evaluation_report, save_predictions
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel
import json
import yaml

print("✅ Setup complete")

In [ ]:
model_path = 'models/final'

print(f"Loading model from: {model_path}")

tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
model = PeftModel.from_pretrained(base_model, model_path)
model.eval()

print("✅ Model loaded and ready for evaluation!")

In [ ]:
with open('data/processed/augmented_train.json', 'r') as f:
    test_data = json.load(f)

print(f"Loaded {len(test_data)} test examples")

In [ ]:
print("Running evaluation...")

metrics, predictions = evaluate_model(
    model=model,
    tokenizer=tokenizer,
    test_data=test_data,
    src_lang="eng_Latn",
    tgt_lang="sin_Sinh"
)

print_evaluation_report(metrics)

In [ ]:
save_predictions(
    predictions=predictions,
    references=[ex['target_si'] for ex in test_data],
    source_texts=[ex['source_en'] for ex in test_data],
    output_path='outputs/full_evaluation.json'
)

print("✅ Results saved to outputs/full_evaluation.json")